# Knihovna OpenPyXL

V této lekci se naučíme pracovat s Excel soubory pomocí knihovny `openpyxl`. Tato knihovna umožňuje vytvářet, otevírat a upravovat Excel soubory ve formátu `.xlsx`.

## Instalace a import knihovny

Pokud nemáte knihovnu nainstalovanou, spusťte:
```
pip install openpyxl
```

In [ ]:
import openpyxl

## Hello World - vytvoření nového Excel souboru

Základní kroky pro vytvoření souboru:
1. Vytvořit objekt `Workbook`
2. Vybrat aktivní list
3. Zapsat data do buněk
4. Uložit soubor

In [ ]:
# Vytvoříme nový Workbook objekt
wb = openpyxl.Workbook()

In [ ]:
# Vybereme aktivní list
ws = wb.active

In [ ]:
# Zapíšeme data do buňky A1
ws['A1'] = 'Hello world'

In [ ]:
# Alternativní způsob zápisu do buňky
ws.cell(row=2, column=1, value='Druhý řádek')

In [ ]:
# Uložíme soubor
wb.save('hello_world.xlsx')

### Otázka k zamyšlení
Jaký je rozdíl mezi zápisem `ws['A1'] = hodnota` a `ws.cell(row=1, column=1, value=hodnota)`?

## Čtení dat z existujícího souboru

Pro otevření existujícího souboru použijeme funkci `load_workbook()`.

In [ ]:
# Otevřeme existující soubor
wb = openpyxl.load_workbook('./hello_world.xlsx')

In [ ]:
# Zobrazíme seznam dostupných listů
print(wb.sheetnames)

In [ ]:
# Vybereme konkrétní list podle jména
ws = wb['Sheet']

In [ ]:
# Přečteme hodnotu z buňky A1
print(ws['A1'].value)

### Úloha: Oprav chybu v kódu

Následující kód obsahuje chybu. Najdi ji a oprav.

In [ ]:
# OPRAV CHYBU:
wb = openpyxl.load_workbook('./hello_world.xlsx')
ws = wb.active
hodnota = ws['A1']  # Tady chybí něco...
print(hodnota)

## Práce s DataFrame

OpenPyXL nabízí funkci `dataframe_to_rows()` pro snadný převod DataFrame na řádky, které lze zapsat do Excelu.

In [ ]:
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
# Vytvoříme DataFrame s informacemi o autech
df = pd.DataFrame(
    [['fiat 126p', 105, 630], 
     ['golf II', 148, 920], 
     ['tico', 143, 670]],
    columns=['model', 'max_speed', 'weight']
)

In [ ]:
print(df)

In [ ]:
# Zapíšeme DataFrame do Excelu
wb = openpyxl.Workbook()
ws = wb.active

for car in dataframe_to_rows(df, index=False, header=True):
    ws.append(car)  # Přidá řádek na konec listu

wb.save('cars.xlsx')

Parametry funkce `dataframe_to_rows()`:
- `df` - DataFrame, který chceme uložit
- `index` - zda chceme uložit index (výchozí: True)
- `header` - zda chceme uložit záhlaví sloupců (výchozí: True)

### Otázka k zamyšlení
Co se stane, když nastavíme `index=True`? Zkuste to a podívejte se na výsledný soubor.

## Filtry a zobrazení dat

OpenPyXL umožňuje definovat filtry přímo v Excel souboru. Filtry se však aplikují až při otevření souboru v Excelu.

In [ ]:
# Příklad filtrace - zobrazí pouze 'tico'
wb = openpyxl.Workbook()
ws = wb.active

for car in dataframe_to_rows(df, index=False, header=True):
    ws.append(car)

In [ ]:
# Nastavíme rozsah dat pro filtr
ws.auto_filter.ref = 'A1:C4'

In [ ]:
# Přidáme filtr na první sloupec (index 0)
ws.auto_filter.add_filter_column(
    col_id=0,           # Sloupec, podle kterého filtrujeme
    vals=['tico']       # Hodnoty, které chceme zobrazit
)

In [ ]:
wb.save('./cars_filtered.xlsx')

## Vytváření nových listů

Pro vytvoření nového listu použijeme metodu `create_sheet()`.

In [ ]:
wb = openpyxl.Workbook()

# Vytvoříme nový list s názvem a pozicí
ws1 = wb.create_sheet('Data', 0)      # První pozice
ws2 = wb.create_sheet('Souhrn', 1)    # Druhá pozice

In [ ]:
print(wb.sheetnames)

In [ ]:
# Zapíšeme data do jednotlivých listů
ws1['A1'] = 'Toto je list Data'
ws2['A1'] = 'Toto je list Souhrn'

wb.save('./vice_listu.xlsx')

### Úloha: Oprav chybu v kódu

Následující kód se snaží vytvořit nový list, ale obsahuje chybu.

In [ ]:
# OPRAV CHYBU:
wb = openpyxl.Workbook()
ws = wb.create_sheet(1, 'MujList')  # Parametry jsou v nesprávném pořadí
ws['A1'] = 'Test'
wb.save('./test.xlsx')

---

# Cvičení

## Cvičení 1: Konverze souboru

Pomocí knihovny `openpyxl` převeďte soubor **product_prices_cleaned.csv** na Excel soubor.

In [ ]:
# Váš kód zde:


## Cvičení 2: Rozdělení dat do listů

Upravte předchozí cvičení tak, aby každá skupina produktů (`product_group_id`) byla v samostatném listu. Například `product_group_id` 1 by mělo být v listu s názvem `1`.

Použijte metodu:
```python
wb.create_sheet(name, index)
```
Kde:
- `name` - název listu (musí být řetězec)
- `index` - pozice listu v sešitu

In [ ]:
# Váš kód zde:


## Cvičení 3: Automatizace pro analytiky

Tým analytiků připravuje měsíční report o cenách produktů vybraných vedením. Požádali vás o automatizaci procesu. Po rozhovoru s týmem jste stanovili následující podmínky:

**Parametry reportu:**
- `product_group_id`
- `product`
- `date`

**Pravidla pro parametry:**
1. Parametr může mít nejvýše jednu hodnotu
2. Pokud je parametr prázdný, vrátíme všechny záznamy ze skupiny
3. Předpokládáme, že soubor je vždy správně připravený

**Úkoly:**
1. Načtěte soubor **config.xlsx** pomocí `openpyxl`
2. Připravte podmínky pro filtrování dat z **product_cleaned.csv**
3. Aplikujte filtry na DataFrame
4. Agregujte data pomocí `pivot_table`:
   - index: product, province
   - columns: dates
   - value: průměrná cena produktu
   - nezapomeňte odstranit nuly
5. Uložte výsledek do Excel souboru

**Tipy:**
- Podmínky filtrování můžete uložit do proměnných a pak je kombinovat: `df.loc[var1 & var2]`
- Při ukládání pomocí Pandas dejte pozor na parametr `index` - co se stane když nastavíte `index=False`?

In [ ]:
# Váš kód zde:


---

# Přehled použitých metod a funkcí

| Metoda/Funkce | Popis |
|---------------|-------|
| `openpyxl.Workbook()` | Vytvoří nový Excel sešit |
| `openpyxl.load_workbook(cesta)` | Otevře existující Excel soubor |
| `wb.active` | Vrátí aktivní list sešitu |
| `wb.sheetnames` | Seznam názvů všech listů |
| `wb.create_sheet(name, index)` | Vytvoří nový list s daným názvem na dané pozici |
| `wb.save(cesta)` | Uloží sešit do souboru |
| `wb['NazevListu']` | Vybere list podle názvu |
| `ws['A1']` | Přístup k buňce pomocí adresy |
| `ws['A1'].value` | Přečte hodnotu z buňky |
| `ws.cell(row, column, value)` | Přístup k buňce pomocí čísel řádku a sloupce |
| `ws.append(seznam)` | Přidá řádek dat na konec listu |
| `ws.auto_filter.ref` | Nastaví rozsah pro automatický filtr |
| `ws.auto_filter.add_filter_column(col_id, vals)` | Přidá filtr na sloupec |
| `dataframe_to_rows(df, index, header)` | Převede DataFrame na řádky pro zápis do Excelu |